- Given a lat lon
- Assume maximum temperature
- convert lat lon to correct projection
- join all times together
- from megacube extract that geospatial point
    - investgate interopolation vs. find nearest
- collapse time by mean
- return float

In [4]:
lat = 52
lon = -2

In [5]:
import iris
iris.FUTURE.netcdf_promote = True

cubes = iris.load('../microsoft/*.nc')
full_cube = iris.cube.CubeList(cubes).concatenate_cube()
full_cube

<iris 'Cube' of air_temperature / (degC) (time: 366; projection_y_coordinate: 290; projection_x_coordinate: 180)>

In [6]:
import cartopy

def transform_coords(cube, lon, lat):
    expected_proj = cube.coords('projection_y_coordinate')[0].coord_system.as_cartopy_crs()
    given_proj = cartopy.crs.PlateCarree()
    return expected_proj.transform_point(lon, lat, given_proj)

In [7]:
x, y = transform_coords(full_cube, lon, lat)
x, y

(400000.0, 233553.7313316085)

In [8]:
first_year = next(full_cube.slices_over('time'))
first_year

<iris 'Cube' of air_temperature / (degC) (projection_y_coordinate: 290; projection_x_coordinate: 180)>

In [9]:
samples = [('projection_y_coordinate', y), ('projection_x_coordinate', x)]
first_point = first_year.interpolate(samples, iris.analysis.Nearest())
print(first_point)

air_temperature / (degC)            (scalar cube)
     Scalar coordinates:
          latitude: 51.9905194856 degrees
          longitude: -2.03641212568 degrees
          projection_x_coordinate: 400000.0 m
          projection_y_coordinate: 233553.731332 m
          time: 1960-01-01 12:00:00, bound=(1960-01-01 00:00:00, 1960-01-02 00:00:00)
     Attributes:
          Conventions: CF-1.5
          comment: This dataset has been produced by Met Office in support of UKCP09
          institution: Met Office
          references: doi: 10.1002/joc.1161
          short_name: daily_maxtemp
          source: UKCP09
          title: UKCP09: Met Office Gridded land surface climate observations
     Cell methods:
          maximum: time (observing period is 0900 on day D to 0900 on day D+1)


In [10]:
exact_x = first_point.coord('projection_x_coordinate')[0].points[0]
exact_y = first_point.coord('projection_y_coordinate')[0].points[0]
exact_x, exact_y

(400000.0, 233553.73133160849)

In [11]:
x_const = iris.Constraint(projection_x_coordinate=exact_x)
y_const = iris.Constraint(projection_y_coordinate=exact_y)

all_points = full_cube.extract(x_const).extract(y_const)
all_points

<iris 'Cube' of air_temperature / (degC) (time: 366)>

In [39]:
years = [1960]

In [40]:
import datetime

time_units = all_points.coord('time').units
vals = [time_units.date2num(datetime.datetime(y, 1, 15)) for y in years]
time_constraint = iris.Constraint(time=vals)
time_constraint

Constraint(coord_values={'time': [1402848.0]})

In [41]:
day_cube = all_points.extract(time_constraint)

In [42]:
time_mean = day_cube.collapsed('time', iris.analysis.MEAN)

In [43]:
print(time_mean)

air_temperature / (degC)            (scalar cube)
     Scalar coordinates:
          latitude: 51.9905194856 degrees
          longitude: -2.03641212568 degrees
          projection_x_coordinate: 397500.0 m, bound=(395000.0, 400000.0) m
          projection_y_coordinate: 232500.0 m, bound=(230000.0, 235000.0) m
          time: 1960-01-15 00:00:00, bound=(1960-01-14 00:00:00, 1960-01-16 00:00:00)
     Attributes:
          Conventions: CF-1.5
          comment: This dataset has been produced by Met Office in support of UKCP09
          institution: Met Office
          references: doi: 10.1002/joc.1161
          short_name: daily_maxtemp
          source: UKCP09
          title: UKCP09: Met Office Gridded land surface climate observations
     Cell methods:
          maximum: time (observing period is 0900 on day D to 0900 on day D+1)
          mean: time


In [44]:
print(time_mean.data)

1.4383695125579834
